 Use SocialGNN_encoding conda environment

In [1]:
from model_functions import SocialGNN, get_inputs_outputs
from LSTM_model_functions import CueBasedLSTM, get_inputs_outputs_baseline

from collections import namedtuple
import pickle
import os 

from tqdm import tqdm

In [2]:
options = ['LSTM', 'SocialGNN', 'LSTM_Relation']
data_input_dir = 'Data/preprocess/graphs'

In [3]:
def load_pickle(path):
    with open(path, 'rb') as f:
        pickled = pickle.load(f)
    return pickled

def save_pickle(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f)

In [4]:
videos = load_pickle(data_input_dir)

In [5]:
def load_parameters(model_selection, trained_model_dir):
    model_list = sorted(os.listdir(trained_model_dir))
    sorted_model_list = []
    if model_selection == 'SocialGNN':
        model_config = namedtuple('model_config', 'NUM_NODES NUM_AGENTS V_SPATIAL_SIZE E_SPATIAL_SIZE V_TEMPORAL_SIZE V_OUTPUT_SIZE BATCH_SIZE CLASS_WEIGHTS LEARNING_RATE LAMBDA')
        for name in model_list:
            if "SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01" in name:
                sorted_model_list.append(name)
        for name in model_list:
            if "SocialGNN_V_5_5_12_12_6_5_20_5.69_4.42_1.85_1.66_1.0_0.001_0.01" in name:
                sorted_model_list.append(name)
    elif model_selection == 'LSTM':
        model_config = namedtuple('model_config', 'FEATURE_SIZE V_TEMPORAL_SIZE V_OUTPUT_SIZE BATCH_SIZE CLASS_WEIGHTS LEARNING_RATE LAMBDA')
        for name in model_list:
            if "CueBasedLSTM_125_6_2_20_1.0_1.5_0.001_0.01" in name:
                sorted_model_list.append(name)
        for name in model_list:
            if "CueBasedLSTM_125_6_5_20_5.69_4.42_1.85_1.66_1.0_0.001_0.01" in name:
                sorted_model_list.append(name)
    elif model_selection == 'LSTM_Relation':
        model_config = namedtuple('model_config', 'FEATURE_SIZE V_TEMPORAL_SIZE V_OUTPUT_SIZE BATCH_SIZE CLASS_WEIGHTS LEARNING_RATE LAMBDA')
        for name in model_list:
            if 'CueBasedLSTM-Relation_145_6_2_20_1.0_1.5_0.001_0.01' in name:
                sorted_model_list.append(name)
        for name in model_list:
            if 'CueBasedLSTM-Relation_145_6_5_20_5.69_4.42_1.85_1.66_1.0_0.001_0.01' in name:
                sorted_model_list.append(name)
    return model_config, sorted_model_list

In [6]:
def save_layer_representations(model_selection):
    trained_model_dir = f'TrainedModels/{model_selection}'
    encoding_output_dir = f'Data/layer_representations/{model_selection}'
    model_config, sorted_model_list = load_parameters(model_selection, trained_model_dir)
    if model_selection == 'SocialGNN':
        old_output_size = None
        for model_name in tqdm(sorted_model_list):
            model_dir = os.path.join(trained_model_dir, model_name)
            if os.path.isdir(model_dir):
                parameters = model_name.split('_')[7:]
                config = model_config(
                    NUM_NODES = 5,              # always 5,
                    NUM_AGENTS = 5,             # always 5
                    V_SPATIAL_SIZE = 12,        # always 12
                    E_SPATIAL_SIZE = 12,        # always 12
                    V_TEMPORAL_SIZE = 6,        # always 6
                    V_OUTPUT_SIZE = int(parameters[5]),      # 2 or 5, only use 5
                    BATCH_SIZE = 10,            # 20, but forced to be 10 so that I don't need to worry about the padded data
                    CLASS_WEIGHTS = [float(parameters[7+i]) for i in range(int(parameters[5]))],
                    LEARNING_RATE = 0.001,      # always 0.001
                    LAMBDA = 0.01               # always 0.01
                )
                if config.V_OUTPUT_SIZE != old_output_size:
                    sample_graph_dicts_list, video_timesteps, _ = get_inputs_outputs(videos)
                    model = SocialGNN(videos, config, sample_graph_dicts_list)
                    old_output_size = config.V_OUTPUT_SIZE
                model.load_model(model_dir)
                layer_representations = model.get_layer_representations()
                
                for key in layer_representations:
                    output_dir = os.path.join(encoding_output_dir, key, model_name)
                    save_pickle(layer_representations[key], output_dir)
                    
    elif model_selection == 'LSTM':
        old_output_size = None
        for model_name in tqdm(sorted_model_list):
            model_dir = os.path.join(trained_model_dir, model_name)
            if os.path.isdir(model_dir):
                
                parameters = model_name.split('_')[6:]
                config = model_config(
                    FEATURE_SIZE = 125,         # always 125
                    V_TEMPORAL_SIZE = 6,        # always 6
                    V_OUTPUT_SIZE = int(parameters[2]),      # 2 or 5
                    BATCH_SIZE = 10,            # 20, but forced to be 10 so that I don't need to worry about the padded data
                    CLASS_WEIGHTS = [float(parameters[4+i]) for i in range(int(parameters[2]))],
                    LEARNING_RATE = 0.001,      # always 0.001
                    LAMBDA = 0.01               # always 0.01
                )
        
        
                if config.V_OUTPUT_SIZE != old_output_size:
                    # sample_graph_dicts_list, video_timesteps, _ = get_inputs_outputs_baseline(videos)
                    model = CueBasedLSTM(videos, config, explicit_edges=False)
                    old_output_size = config.V_OUTPUT_SIZE
                
                model.load_model(model_dir)
                layer_representations = model.get_layer_representations()
                
                for key in layer_representations:
                    output_dir = os.path.join(encoding_output_dir, key, model_name)
                    save_pickle(layer_representations[key], output_dir)  
    elif model_selection == 'LSTM_Relation':
        old_output_size = None
        for model_name in tqdm(sorted_model_list):
            model_dir = os.path.join(trained_model_dir, model_name)
            if os.path.isdir(model_dir):
                
                parameters = model_name.split('_')[6:]
                config = model_config(
                    FEATURE_SIZE = 145,         # always 145
                    V_TEMPORAL_SIZE = 6,        # always 6
                    V_OUTPUT_SIZE = int(parameters[2]),      # 2 or 5
                    BATCH_SIZE = 10,            # 20, but forced to be 10 so that I don't need to worry about the padded data
                    CLASS_WEIGHTS = [float(parameters[4+i]) for i in range(int(parameters[2]))],
                    LEARNING_RATE = 0.001,      # always 0.001
                    LAMBDA = 0.01               # always 0.01
                )
        
        
                if config.V_OUTPUT_SIZE != old_output_size:
                    # sample_graph_dicts_list, video_timesteps, _ = get_inputs_outputs_baseline(videos)
                    model = CueBasedLSTM(videos, config, explicit_edges=True)
                    old_output_size = config.V_OUTPUT_SIZE
                
                model.load_model(model_dir)
                layer_representations = model.get_layer_representations()
                
                for key in layer_representations:
                    output_dir = os.path.join(encoding_output_dir, key, model_name)
                    save_pickle(layer_representations[key], output_dir)  

In [7]:
for model_selection in options:
    save_layer_representations(model_selection)

  0%|          | 0/40 [00:00<?, ?it/s]

.............DEFINING INPUT PLACEHOLDERS..............

.............BUILDING GRAPH..............

.............INITIALIZATION SESSION..............


 50%|█████     | 20/40 [00:15<00:15,  1.26it/s]

.............DEFINING INPUT PLACEHOLDERS..............

.............BUILDING GRAPH..............

.............INITIALIZATION SESSION..............


  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

.............DEFINING INPUT PLACEHOLDERS..............

.............BUILDING GRAPH..............

.............INITIALIZATION SESSION..............


 50%|█████     | 20/40 [01:13<01:11,  3.60s/it]

.............DEFINING INPUT PLACEHOLDERS..............

.............BUILDING GRAPH..............

.............INITIALIZATION SESSION..............


100%|██████████| 40/40 [02:27<00:00,  3.70s/it]
